In [ ]:
import numpy as np
import pandas as pd
import nltk

from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.preprocessing import OneHotEncoder

from sklearn.ensemble import GradientBoostingClassifier

from sklearn.decomposition import TruncatedSVD
pd.options.mode.chained_assignment = None 
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
!wget http://download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz
!tar -xvf mystem-3.0-linux3.1-64bit.tar.gz
!mkdir -p /root/.local/bin/
!cp mystem /root/.local/bin/mystem

from pymystem3 import Mystem

--2020-11-07 22:16:58--  http://download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz
Resolving download.cdn.yandex.net (download.cdn.yandex.net)... 5.45.205.245, 5.45.205.244, 5.45.205.241, ...
Connecting to download.cdn.yandex.net (download.cdn.yandex.net)|5.45.205.245|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: http://cache-mskm904.cdn.yandex.net/download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz [following]
--2020-11-07 22:16:58--  http://cache-mskm904.cdn.yandex.net/download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz
Resolving cache-mskm904.cdn.yandex.net (cache-mskm904.cdn.yandex.net)... 5.45.220.14, 2a02:6b8:0:2002::15
Connecting to cache-mskm904.cdn.yandex.net (cache-mskm904.cdn.yandex.net)|5.45.220.14|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16457938 (16M) [application/octet-stream]
Saving to: ‘mystem-3.0-linux3.1-64bit.tar.gz.3’

mystem-3.0-linux3.1 100%[===================>] 

In [ ]:
train_df = pd.read_csv("train.csv")

train_df.head()

,id,url,title,target
0,0,m.kp.md,"Экс-министр экономики Молдовы - главе МИДЭИ, ц...",False
1,1,www.kp.by,Эта песня стала известна многим телезрителям б...,False
2,2,fanserials.tv,Банши 4 сезон 2 серия Бремя красоты смотреть о...,False
3,3,colorbox.spb.ru,Не Беси Меня Картинки,False
4,4,tula-sport.ru,В Новомосковске сыграют следж-хоккеисты алекси...,False


In [ ]:
test_df = pd.read_csv("test.csv")

test_df.head()

,id,url,title
0,135309,www.kommersant.ru,Шестой кассационный суд в Самаре начнет работу...
1,135310,urexpert.online,"Что такое индексация алиментов, кем и в каких ..."
2,135311,imperimeha.ru,Женщинам | Империя Меха - Part 12
3,135312,national-porn.com,"Небритые, волосатые киски: Порно всех стран и ..."
4,135313,2gis.ru,67


In [ ]:
train_df["target"].value_counts()

False    118594
True      16715
Name: target, dtype: int64

Следующие две ячейки могут использоваться в конце, чтобы обновить 100% порнушные и чистые домены. Чисто логически это кажется довольно неплохой добавкой, хотя работает жутко медленно и на деле почти не улучшает скор

In [ ]:
porn_urls = train_df[train_df['target'] == True]['url']
porn_urls = set(porn_urls.unique())

In [ ]:
non_porn_urls = train_df[train_df['target'] == False]['url']
non_porn_urls = set(non_porn_urls.unique())

Добавляем самые популярные слова в стоп, чтобы хоть как-то сократить размерность

In [ ]:
from nltk.corpus import stopwords

russian_stop_words = set(stopwords.words('russian') + ['это', '-', 'эта', 'эти', ',', '.', ')', '('
')\n', '(\n', '\n'])
english_stop_words = set(stopwords.words('english'))

stop_words = russian_stop_words | english_stop_words


Нормализуем через MyStem, возможно стоит попробовать еще и другие нормализаторы типа стеммера Портера



In [ ]:

normalizer = Mystem()
stop_chars = ',.;:!?%$^&-]['

def lemmatize(text):
  for char in stop_chars:
    text = text.replace(char, ' ')
  try:
    normalized = normalizer.lemmatize(text)
    normalized_non_stop = [word for word in normalized if word not in stop_words]
    return "".join(normalized_non_stop).strip()  
  except Exception as e:
    print(e)
    return ""

Добавление url к списку всех слов довольно сильно переобучает модель, получается, что модель настраивается в основном на ссылки и не придает значения тексту, поэтому эту часть стоит оставить закомментированной

In [ ]:
train_df['url_title'] = train_df['url'] + ' ' + train_df['title']
test_df['url_title'] = test_df['url'] + ' ' + test_df['title']

In [ ]:
train_df['title_vectorized'] = train_df['url_title'].apply(lemmatize)
test_df['title_vectorized'] = test_df['url_title'].apply(lemmatize)

In [ ]:
X = train_df['title_vectorized'].values
y = train_df['target'].astype(int)
X_test = test_df['title_vectorized'].values

Тут в основном настраивались параметры min_df и ngram_range, среди всех возможных параметров выбранные показали наилучшее сочетание скора и размерности пространства:

(2, 2) - плохой скор

(1, 3) - скор не сильно лучше (или буквально такой же), но размерность растет

(1, 2) - дал самое оптимальное соотношение

Идея по улучшению: некоторые слова могут быть написаны с ошибками,поэтому лучше разбивать не по словам, а по группам символов. Оптимальные параметры (3, 6)

In [ ]:
# stop_words=stop_words, 

vectorizer = TfidfVectorizer(
    lowercase=True, ngram_range=(3, 6), analyzer='char_wb',
    min_df=2, max_df=0.8,
    sublinear_tf=True
)

X_transformed = vectorizer.fit_transform(X)
X_test_transformed = vectorizer.transform(X_test)

In [ ]:
def train_model(model, X_train, X_validate, y_train, y_validate):
  model.fit(X_train, y_train)
  y_pred = model.predict(
      X_validate
  )
  print("F1 score: {}".format(f1_score(y_pred, y_validate)))
  print(classification_report(y_validate, y_pred))


In [ ]:
X_train, X_validate, y_train, y_validate = train_test_split(X_transformed, y,
                                                            test_size=0.33, random_state=42)

Какие модели стоит потестить:

-  LogisticRegression  - из всех решателей был выбран liblinear, class_weight = 'balanced', тюн остальных параметров ничего существенного не дал

- Приделать к логистической регрессии RF SF: по итогу ожидалось, что качество будет расти от числа estimators, но этого не произошло (возможно это связано с обработкой данных и настройкой лемматайзера и стеммера). При этом пробовались как и семплирование фич, так и семплирование наблюдений


- Если останется время попробовать keras или tf?????

Что НЕ стоит тестить:
- Все, что связано с деревьями (плохо будут работать на таких разреженных данных), сюда же наверное стоит добавить и градиент бустинг

- SGDClassifier - потому что это LogisticRegression на минималках

- Наивный байас

In [ ]:
%%time
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier

model = LogisticRegression(solver='liblinear', class_weight='balanced', random_state=42)
# model = RandomForestClassifier(random_state=42)
train_model(model, X_train, X_validate, y_train, y_validate)

F1 score: 0.98131010705861
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     39137
           1       0.98      0.98      0.98      5515

    accuracy                           1.00     44652
   macro avg       0.99      0.99      0.99     44652
weighted avg       1.00      1.00      1.00     44652

CPU times: user 8.77 s, sys: 4.96 s, total: 13.7 s
Wall time: 8.01 s


In [ ]:
%%time

test_df["target"] = model.predict(X_test_transformed).astype(bool)
for url in porn_urls:
  test_df[(test_df['url'] == url) & (test_df['target'] == False)]['target'] = True
for url in non_porn_urls:
  test_df[(test_df['url'] == url) & (test_df['target'] == True)]['target'] = False
test_df[["id", "target"]].to_csv("ml_baseline.csv", index=False)


CPU times: user 7min 6s, sys: 1.28 s, total: 7min 7s
Wall time: 7min 9s


Идеи:

Настроить умное приведение к нижнему регистру - Dick Hammigton классифицирует как порно, а не как человека( (не сделано)

Разбивать не по словам, а по n символам, чтобы обработать ошибки в тайтлах по типу гомосексулст, порн, и тп (влечет за собой проблемы из графы Не все так однозначно) (сделано)



### Не все так однозначно

**не порно**:
- Болезни опорно-двигательной системы и импотенция: взаимосвязь
- Транссексуальные рыбы - National Geographic Россия: красота мира в каждом кадре
- Групповая обзорная экскурсия по Афинам - цена €50
- Больного раком Задорнова затравили в соцсетях.
- Гомосексуалисты на «Первом канале»? Эрнст и Галкин – скрытая гей-пара российского шоу-бизнеса | Заметки о стиле, моде и жизни

**порно**:
- Отборная домашка
- Сюзанна - карьера горничной / Susanna cameriera perversa (с русским переводом) 1995 г., DVDRip